<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
    <h1 align="center">Thesis - Bias MAE</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import mae
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # train the algorithm with the training dataset
    # Now we run the recommender:
    preds = batch.predict(fittable, test)
    # add the algorithm name for analyzability
    return preds

In [2]:
def model_trainer(item,user,damping):
    all_recs = []
    test_data = []
    bias = Bias(items=item, users=user, damping=damping) # define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('Bias', bias, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)

    metric = mae(all_recs['prediction'], all_recs['rating'])
    
    return metric

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    user = params['user']
    item = params['item']
    damping = params['damping']
    metric = model_trainer(item,user,damping)
    print([item,user,damping,metric],',')
    return {'loss': metric, 'status': STATUS_OK }

space={'item': hp.choice('item', ['True', 'False']),
       'user': hp.choice('user', ['True', 'False']),
       'damping': hp.uniform('damping', 0, 1)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

['True', 'True', 0.7632925115864152, 0.7477532574945027]                                                               
,                                                                                                                      
['True', 'True', 0.2995301386553929, 0.7472082234590165]                                                               
,                                                                                                                      
['False', 'False', 0.6419550491667957, 0.7397411973703452]                                                             
,                                                                                                                      
['True', 'False', 0.5900397767599559, 0.7482764370768992]                                                              
,                                                                                                                      
['False', 'True', 0.022889937743149735, 

['False', 'False', 0.9801092630093282, 0.7438551138650973]                                                             
,                                                                                                                      
['False', 'False', 0.3029448911572252, 0.7520922973150829]                                                             
,                                                                                                                      
['False', 'False', 0.6643485395047828, 0.7446971755636791]                                                             
,                                                                                                                      
['False', 'False', 0.39209871858074946, 0.7403007176996207]                                                            
,                                                                                                                      
['False', 'False', 0.7976520559271875, 0

['False', 'True', 0.1502157701191058, 0.7487338683758357]                                                              
,                                                                                                                      
['False', 'True', 0.4356458614188002, 0.7446199875983842]                                                              
,                                                                                                                      
['False', 'True', 0.36146876462344507, 0.744092528581961]                                                              
,                                                                                                                      
['False', 'True', 0.23449067761768883, 0.7514059180987988]                                                             
,                                                                                                                      
['False', 'True', 0.10083800833424966, 0

In [11]:
metric = pd.DataFrame(array)
metric.columns = ['items','users','damping','MAE']
metric.head(5)

,items,users,damping,MAE
0,True,True,0.763293,0.747753
1,True,True,0.299530,0.747208
2,False,False,0.641955,0.739741
3,True,False,0.590040,0.748276
4,False,True,0.022890,0.750046


In [13]:
metric.sort_values(by=['MAE'],ascending=True).head(10)

,items,users,damping,MAE
81,True,False,0.588636,0.735289
25,False,False,0.366823,0.736717
23,False,False,0.993655,0.736936
6,False,True,0.351006,0.737358
30,False,False,0.299633,0.737807
59,False,False,0.168904,0.738815
78,False,False,0.411010,0.739066
2,False,False,0.641955,0.739741
51,False,True,0.559990,0.739912
37,False,False,0.392099,0.740301
